
# Phase 2 – Function Calling & Tooling (LangChain / OpenAI)
**Mục tiêu:** Hiểu rõ sự khác nhau giữa *Function Calling* và *Tooling*, kèm ví dụ chạy thực tế, schema, retry & validate, và best practices.

> **Gợi ý chạy:** tạo virtualenv, cài `langchain`, `langchain-openai`, `pydantic`, `python-dotenv`.



## 1) Tổng quan: Function Calling vs Tooling
- **Function Calling:** LLM hiểu *schema* của hàm và sinh JSON arguments hợp lệ. (Giống như *“biết gọi đúng chữ ký hàm”*).
- **Tooling:** LLM được gắn *các công cụ thực thi thật*; runtime quản lý gọi hàm, validate, retry, log, và phối hợp nhiều tool theo ngữ cảnh.

**Sơ đồ dòng chảy (tư duy):**
```
Python Function
     ↓
Type Hints + Field(...)  ←─── context có cấu trúc
     ↓
Pydantic / JSON Schema
     ↓
LLM Tool Description
     ↓
LLM hiểu đúng tham số → Gọi tool → Nhận output thật → Tiếp tục reasoning
```



## 2) Chuẩn bị môi trường
- Đặt OpenAI API key vào `.env` hoặc biến môi trường.
- Cài thư viện:
```bash
pip install -U langchain langchain-openai pydantic python-dotenv
```


In [1]:

# ⚙️ Thiết lập env (tuỳ chọn). Không bắt buộc chạy nếu bạn xuất API key theo cách khác.
# phụ chú: Dùng dotenv để tự động nạp OPENAI_API_KEY từ file .env khi chạy notebook này.
import os
from pathlib import Path

try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception as e:
    print("dotenv chưa được cài đặt hoặc không cần thiết.", e)

print("OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))


OPENAI_API_KEY set: True



## 3) Function Calling – Structured Output cơ bản (không Tool)
Ở ví dụ này, ta dùng **structured output** (schema Pydantic) để buộc LLM trả về đúng cấu trúc — đây là tinh thần của *Function Calling*.


In [2]:

# phụ chú: Ví dụ structured output với Pydantic để LLM trả về JSON đúng schema.
from pydantic import BaseModel, Field
from typing import List

class Intro(BaseModel):
    title: str = Field(..., description="Tiêu đề ngắn gọn")
    bullets: List[str] = Field(..., description="Các ý chính, dạng danh sách")

# phụ chú: Nếu dùng langchain-openai, có thể dùng with_structured_output:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
# structured_llm = llm.with_structured_output(Intro)
# out: Intro = structured_llm.invoke("Hãy viết phần giới thiệu về LangChain.")
# print(out)

# Lưu ý: Cell này chỉ nêu schema. Chạy thực tế cần cài langchain-openai và có API key.
Intro.model_json_schema()


{'properties': {'title': {'description': 'Tiêu đề ngắn gọn',
   'title': 'Title',
   'type': 'string'},
  'bullets': {'description': 'Các ý chính, dạng danh sách',
   'items': {'type': 'string'},
   'title': 'Bullets',
   'type': 'array'}},
 'required': ['title', 'bullets'],
 'title': 'Intro',
 'type': 'object'}


## 4) Tooling – Tạo tool bằng `@tool` (schema tự sinh)
Dùng `@tool` để bọc hàm Python thành **StructuredTool**. `Field(...)` giúp *đưa context có cấu trúc* cho LLM.


In [4]:

# phụ chú: Demo @tool với Annotated + Field để ràng buộc/diễn giải tham số.
from typing import Annotated, Optional
from pydantic import Field
from langchain.tools import tool

@tool
def get_exchange_rate(
    pair: Annotated[str, Field(description="Cặp tiền tệ dạng 'USD/VND', 'EUR/VND'")],
    source: Annotated[Optional[str], Field(description="Nguồn tỷ giá: demo/ecb/fiin")] = "demo",
) -> dict:
    """Tra cứu tỷ giá giữa 2 đồng tiền."""
    rates = {"USD/VND": 24500, "EUR/VND": 26600}
    return {"pair": pair.upper(), "rate": rates.get(pair.upper()), "source": source}

# phụ chú: In schema để thấy LLM sẽ được cung cấp gì khi quyết định gọi tool.
schema = get_exchange_rate.args_schema.model_json_schema()  # Pydantic v1 style; v2 dùng model_json_schema()
print(schema)


{'description': 'Tra cứu tỷ giá giữa 2 đồng tiền.', 'properties': {'pair': {'description': "Cặp tiền tệ dạng 'USD/VND', 'EUR/VND'", 'title': 'Pair', 'type': 'string'}, 'source': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': 'demo', 'description': 'Nguồn tỷ giá: demo/ecb/fiin', 'title': 'Source'}}, 'required': ['pair'], 'title': 'get_exchange_rate', 'type': 'object'}



## 5) Tooling – `StructuredTool.from_function(..., args_schema=...)`
Khi cần kiểm soát chặt chẽ: validators, alias, examples…, dùng Pydantic `BaseModel` làm `args_schema`.


In [7]:

from pydantic import BaseModel, Field, ValidationError
from typing import Literal
from langchain_core.tools import StructuredTool

class RateArgs(BaseModel):
    pair: str = Field(..., description="Cặp tiền tệ 'BASE/QUOTE', ví dụ 'USD/VND'")
    source: Literal["demo", "ecb", "fiin"] = Field("demo", description="Nguồn tỷ giá")

def get_fx(pair: str, source: str) -> dict:
    rates = {"USD/VND": 24500, "EUR/VND": 26600}
    return {"pair": pair.upper(), "rate": rates.get(pair.upper()), "source": source}

get_fx_tool = StructuredTool.from_function(
    func=get_fx,
    name="get_exchange_rate_strict",
    description="Tra cứu tỷ giá theo cặp tiền tệ từ nguồn chỉ định (schema chặt).",
    args_schema=RateArgs,
)

print(get_fx_tool.args_schema.model_json_schema())


{'properties': {'pair': {'description': "Cặp tiền tệ 'BASE/QUOTE', ví dụ 'USD/VND'", 'title': 'Pair', 'type': 'string'}, 'source': {'default': 'demo', 'description': 'Nguồn tỷ giá', 'enum': ['demo', 'ecb', 'fiin'], 'title': 'Source', 'type': 'string'}}, 'required': ['pair'], 'title': 'RateArgs', 'type': 'object'}



## 6) Gắn tool vào LLM & gọi thực thi (orchestration)
Phần này minh hoạ cách LLM chọn tool, tạo JSON arguments, và runtime thực thi tool để trả output.


In [ ]:
# phụ chú: Ví dụ bind_tools; cần langchain-openai và có API key để chạy thật.
# Nếu chưa cài thư viện/đặt key, bạn có thể bỏ qua cell này.
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_tools = llm.bind_tools([get_exchange_rate, get_fx_tool])

query = "Cho tôi tỷ giá USD/VND hôm nay và nguồn ecb."
resp = llm_tools.invoke(query)  # phụ chú: LangChain sẽ tự động cho LLM chọn tool và gọi thật.
print(resp)


content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 185, 'total_tokens': 209, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CYnE2vzgSUCGulrIAqXhZAdWT7dWV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--03adeab2-b694-4963-a202-84c5e7057a63-0' tool_calls=[{'name': 'get_exchange_rate_strict', 'args': {'pair': 'USD/VND', 'source': 'ecb'}, 'id': 'call_poLRuKJear6v0Wqb5ov4nuBm', 'type': 'tool_call'}] usage_metadata={'input_tokens': 185, 'output_tokens': 24, 'total_tokens': 209, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [10]:
print(resp.tool_calls)

[{'name': 'get_exchange_rate_strict', 'args': {'pair': 'USD/VND', 'source': 'ecb'}, 'id': 'call_poLRuKJear6v0Wqb5ov4nuBm', 'type': 'tool_call'}]



## 7) Validate, lỗi & Retry (rất quan trọng trong Tooling)
- Nếu LLM truyền tham số sai (ví dụ `pair="USDVND"` không có `/`), Pydantic sẽ ném lỗi.
- Tool runtime có thể để LLM *tự sửa* hoặc bạn viết cơ chế retry thủ công.


In [11]:

# phụ chú: Mô phỏng validate lỗi bằng cách gọi schema trực tiếp.
try:
    RateArgs(pair="USDVND", source="ecb")  # thiếu dấu '/'
except ValidationError as e:
    print("❌ ValidationError:", e)

# phụ chú: Ví dụ đúng
ok = RateArgs(pair="USD/VND", source="ecb")
print("✅ Parsed:", ok.model_dump())


✅ Parsed: {'pair': 'USD/VND', 'source': 'ecb'}



## 8) Best Practices (nhớ để production)
- **Đặt `description` & `examples` trong `Field(...)`** → LLM hiểu tham số tốt hơn.
- **Dùng `Literal` / `Enum` cho giá trị ràng buộc** → giảm lỗi.
- **Validators** trong BaseModel để kiểm tra pattern/phạm vi (vd: cặp `BASE/QUOTE`).
- **Nhất quán kiểu trả về** (dict/str) → dễ render & test.
- **Log & Telemetry**: ghi `tool_name`, `args`, `duration`, `errors`.
- **Retry** có kiểm soát: chỉ retry khi lỗi “sửa được” (schema sai, thiếu arg).
- **Tách tool “đọc dữ liệu” và tool “hiển thị/ghi dữ liệu”** để dễ debug.



## 9) Bài tập nhỏ
1) Viết tool `report_statement(symbol, statement, year)` với `Literal["BS","IS","CF"]`, `Field(...)` có ví dụ.  
2) Viết model `StatementArgs` (Pydantic) và tạo `StructuredTool.from_function`.  
3) Gắn cả 2 tool (phiên bản nhanh `@tool` và chặt `args_schema`) vào LLM, đặt câu hỏi:  
   > "Cho tôi Báo cáo KQKD (IS) của HPG năm 2022"  
   Quan sát LLM chọn tool nào, và arguments sinh ra ra sao.  
4) Cố tình truyền input sai để xem lỗi validate hoạt động như thế nào, sau đó sửa prompt để LLM tự gọi đúng.



---
### Kết luận
- **Function Calling**: cho LLM *khả năng hiểu và sinh payload JSON hợp lệ* theo schema.  
- **Tooling**: thêm lớp *thực thi thật*, *validate, retry, log*, và *phối hợp nhiều tool* → gần với *agent thực chiến*.

> Tiếp theo: bạn có thể mở rộng sang **planning / multi-step tools** với LangGraph hoặc dùng **MCP** để expose tool ra ngoài process.
